# Modellbewertung mit Iris Dataset - Versicolor vs. Rest 🌸

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_18/model_evaluation_iris_versicolor_vs_rest.ipynb)

## Willkommen zur vollständigen Modellbewertung!

Heute lernen wir, wie man Machine Learning Modelle **ehrlich und aussagekräftig** bewertet. Wir verwenden das Iris-Dataset und machen daraus ein binäres Klassifikationsproblem: **"Ist es eine Versicolor oder nicht?"**

### Was wir heute lernen:
1. 🔄 **Train/Test-Split** - Ehrliche Bewertung ohne Data Leakage
2. 🔀 **Cross-Validation** - Robuste Performance-Schätzung
3. 📊 **Konfusionsmatrix** - Wo macht unser Modell Fehler?
4. 📈 **Metriken verstehen** - Accuracy, Precision, Recall, F1-Score
5. 📉 **ROC-Kurven** - Schwellenwert-Optimierung
6. ⚖️ **Anwendungskontext** - Wann welche Metrik wichtig ist

**Los geht's!** 🚀

In [ ]:
# Schritt 1: Alle benötigten Bibliotheken importieren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    confusion_matrix, classification_report, 
    accuracy_score, precision_score, recall_score, f1_score,
    roc_curve, roc_auc_score, precision_recall_curve
)
import warnings
warnings.filterwarnings('ignore')

# Für schönere Plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("🎉 Alle Bibliotheken erfolgreich geladen!")
print("Wir sind bereit für professionelle Modellbewertung!")

## 🌺 Schritt 1: Daten laden und zu binärem Problem umwandeln

Das Iris-Dataset hat 3 Klassen (Setosa, Versicolor, Virginica). Wir machen daraus ein **binäres Problem**: Versicolor vs. alle anderen.

**Warum Versicolor?** Versicolor ist schwieriger zu unterscheiden als Setosa - perfekt um zu lernen, wie Modelle mit herausfordernderen Problemen umgehen!

In [ ]:
# Iris-Dataset laden
iris = load_iris()
X = iris.data
y_multiclass = iris.target  # Original: 0=Setosa, 1=Versicolor, 2=Virginica

# Zu binärem Problem umwandeln: Versicolor (1) vs. Rest (0)
y_binary = (y_multiclass == 1).astype(int)  # 1 wenn Versicolor, 0 sonst

# Als DataFrame für bessere Übersicht
df = pd.DataFrame(X, columns=iris.feature_names)
df['Original_Class'] = [iris.target_names[i] for i in y_multiclass]
df['Binary_Target'] = y_binary
df['Binary_Label'] = df['Binary_Target'].map({1: 'Versicolor', 0: 'Not_Versicolor'})

print(f"📊 Dataset-Info:")
print(f"   Anzahl Blumen: {len(df)}")
print(f"   Features: {list(iris.feature_names)}")
print(f"\n🎯 Binäre Klassenverteilung:")
print(df['Binary_Label'].value_counts())
print(f"\n📈 Prozentuale Verteilung:")
print(df['Binary_Label'].value_counts(normalize=True) * 100)

# Erste 5 Zeilen anzeigen
print(f"\n🔍 Erste 5 Datensätze:")
df.head()

## 📊 Datenvisualisierung - Können wir die Klassen unterscheiden?

Schauen wir uns an, ob Versicolor gut von den anderen unterscheidbar ist:

In [ ]:
# Visualisierung der Features
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Iris Features: Versicolor vs. Rest', fontsize=16, fontweight='bold')

features = iris.feature_names
colors = ['red', 'blue']
labels = ['Versicolor', 'Not Versicolor']

for i, feature in enumerate(features):
    ax = axes[i//2, i%2]
    
    # Histogramme für beide Klassen
    for class_val, color, label in zip([1, 0], colors, labels):
        data = df[df['Binary_Target'] == class_val][feature]
        ax.hist(data, alpha=0.7, color=color, label=label, bins=15, edgecolor='black')
    
    ax.set_xlabel(feature)
    ax.set_ylabel('Häufigkeit')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistiken
print("📊 FEATURE-MITTELWERTE:")
for feature in features:
    versicolor_mean = df[df['Binary_Target'] == 1][feature].mean()
    other_mean = df[df['Binary_Target'] == 0][feature].mean()
    print(f"   {feature:20}: Versicolor={versicolor_mean:.2f}, Rest={other_mean:.2f}, Differenz={abs(versicolor_mean-other_mean):.2f}")

print("Achtung! Histogramme der Nicht-Versicolor-Klasse sind teilweise bimodal!")
print("Abstand der Mittelwerte ist ggf. kein gutes Kriterium für Trennbarkeit.")

## 🔄 Schritt 2: Train/Test-Split mit korrekter Stratifizierung

**Wichtig:** Wir teilen die Daten BEVOR wir irgendetwas anderes machen. Die Testdaten bleiben unberührt bis zum Schluss!

In [ ]:
# Train/Test-Split (80/20) mit Stratifizierung
X_train, X_test, y_train, y_test = train_test_split(
    X, y_binary,
    test_size=0.2,          # 20% für Test
    random_state=42,        # Reproduzierbare Ergebnisse
    stratify=y_binary       # Gleiche Klassenverteilung in Train und Test
)

print(f"📊 TRAIN/TEST-SPLIT:")
print(f"   Training: {X_train.shape[0]} Blumen")
print(f"   Test: {X_test.shape[0]} Blumen")
print(f"   Verhältnis: {X_train.shape[0]/X_test.shape[0]:.1f}:1 (Train:Test)")

# Klassenverteilung überprüfen
print(f"\n🎯 KLASSENVERTEILUNG:")
print(f"   Training - Versicolor: {np.sum(y_train)}, Not-Versicolor: {len(y_train) - np.sum(y_train)}")
print(f"   Test - Versicolor: {np.sum(y_test)}, Not-Versicolor: {len(y_test) - np.sum(y_test)}")
print(f"   Training % Versicolor: {np.mean(y_train)*100:.1f}%")
print(f"   Test % Versicolor: {np.mean(y_test)*100:.1f}%")

print(f"\n✅ Stratifizierung erfolgreich - gleiche Verteilung in Train und Test!")

## 🔧 Schritt 3: Pipeline aufbauen (Data Leakage vermeiden!)

**Pipeline = Skalierung + Modell in einem Schritt**

**Warum Pipeline?** Sie verhindert Data Leakage - die Skalierung wird nur aus den Trainingsdaten gelernt!

In [ ]:
# Pipeline erstellen: StandardScaler + LogisticRegression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

print("🔧 PIPELINE AUFGEBAUT:")
print("   1. StandardScaler - Features auf Mittelwert=0, Std=1 skalieren")
print("   2. LogisticRegression - Lineare Klassifikation mit Wahrscheinlichkeiten")
print("\n✅ Data Leakage vermieden - Skalierung nur aus Trainingsdaten!")

# Zusätzlich: Random Forest Pipeline für Vergleich
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42, n_estimators=100))
])

print("\n🌳 ZUSÄTZLICHE PIPELINE für Vergleich:")
print("   Random Forest - Ensemble von Entscheidungsbäumen")

## 🔀 Schritt 4: Cross-Validation - Robuste Performance-Schätzung

**Ein einzelner Train/Test-Split kann Glück oder Pech haben.** 
Cross-Validation wiederholt den Test 5 mal mit verschiedenen Splits!

In [ ]:
# 5-Fold Stratified Cross-Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Verschiedene Metriken testen
scoring_metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

print("🔀 CROSS-VALIDATION ERGEBNISSE (5-Fold):")
print("="*60)

cv_results = {}
for metric in scoring_metrics:
    scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring=metric)
    cv_results[metric] = scores
    
    print(f"{metric.upper():12}: {scores.mean():.3f} ± {scores.std():.3f} | Alle Folds: {[f'{s:.3f}' for s in scores]}")

print("="*60)
print("📊 INTERPRETATION:")
print(f"   Accuracy:  {cv_results['accuracy'].mean():.1%} der Vorhersagen sind korrekt")
print(f"   Precision: {cv_results['precision'].mean():.1%} der 'Setosa'-Vorhersagen stimmen")
print(f"   Recall:    {cv_results['recall'].mean():.1%} aller Setosas werden erkannt")
print(f"   F1-Score:  {cv_results['f1'].mean():.3f} (harmonisches Mittel von Precision & Recall)")
print(f"   ROC-AUC:   {cv_results['roc_auc'].mean():.3f} (0.5=zufällig, 1.0=perfekt)")

# Stabilität bewerten
print(f"\n🎯 MODELL-STABILITÄT:")
for metric in scoring_metrics:
    std_dev = cv_results[metric].std()
    stability = "sehr stabil" if std_dev < 0.02 else "stabil" if std_dev < 0.05 else "relativ stabil" if std_dev < 0.1 else "instabil"
    print(f"   {metric:12}: ±{std_dev:.3f} → {stability}")

## 🏋️ Schritt 5: Modell auf Trainingsdaten trainieren

Jetzt trainieren wir das finale Modell mit allen Trainingsdaten:

In [ ]:
# Modell trainieren
print("🏋️ Trainiere das finale Modell...")
pipeline.fit(X_train, y_train)
rf_pipeline.fit(X_train, y_train)

# Vorhersagen für Testdaten
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]  # Wahrscheinlichkeit für Klasse 1 (Versicolor)

y_pred_rf = rf_pipeline.predict(X_test)
y_pred_proba_rf = rf_pipeline.predict_proba(X_test)[:, 1]

print("✅ Modelle trainiert!")
print(f"\n🔍 FEATURE-WICHTIGKEIT (Logistic Regression):")
feature_importance = np.abs(pipeline.named_steps['classifier'].coef_[0])
feature_ranks = sorted(zip(iris.feature_names, feature_importance), key=lambda x: x[1], reverse=True)
for i, (feature, importance) in enumerate(feature_ranks):
    print(f"   {i+1}. {feature:25}: {importance:.3f}")

print(f"\n🔍 FEATURE-WICHTIGKEIT (Random Forest):")
feature_importance_rf = rf_pipeline.named_steps['classifier'].feature_importances_
feature_ranks = sorted(zip(iris.feature_names, feature_importance_rf), key=lambda x: x[1], reverse=True)
for i, (feature, importance) in enumerate(feature_ranks):
    print(f"   {i+1}. {feature:25}: {importance:.3f}")

## 📊 Schritt 6: Konfusionsmatrix - Wo macht unser Modell Fehler?

Die Konfusionsmatrix zeigt uns **genau**, welche Fehler unser Modell macht:

In [ ]:
# Konfusionsmatrix berechnen
cm = confusion_matrix(y_test, y_pred)
cm_rf = confusion_matrix(y_test, y_pred_rf)

# Schöne Visualisierung
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Logistic Regression
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
           xticklabels=['Not Versicolor', 'Versicolor'], 
           yticklabels=['Not Versicolor', 'Versicolor'], ax=ax1)
ax1.set_title('Konfusionsmatrix - Logistic Regression', fontweight='bold')
ax1.set_xlabel('Vorhersage')
ax1.set_ylabel('Tatsächlich')

# Random Forest
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens', 
           xticklabels=['Not Versicolor', 'Versicolor'], 
           yticklabels=['Not Versicolor', 'Versicolor'], ax=ax2)
ax2.set_title('Konfusionsmatrix - Random Forest', fontweight='bold')
ax2.set_xlabel('Vorhersage')
ax2.set_ylabel('Tatsächlich')

plt.tight_layout()
plt.show()

# Matrix interpretieren
tn, fp, fn, tp = cm.ravel()
tn_rf, fp_rf, fn_rf, tp_rf = cm_rf.ravel()

print("📊 KONFUSIONSMATRIX INTERPRETATION (Logistic Regression):")
print(f"   True Positives (TP):  {tp:2d} - Versicolors korrekt als Versicolor erkannt")
print(f"   True Negatives (TN):  {tn:2d} - Nicht-Versicolors korrekt als Nicht-Versicolor erkannt")
print(f"   False Positives (FP): {fp:2d} - Nicht-Versicolors fälschlich als Versicolor erkannt (Fehlalarm)")
print(f"   False Negatives (FN): {fn:2d} - Versicolors übersehen (verpasste Erkennung)")

print(f"\n📊 KONFUSIONSMATRIX INTERPRETATION (Random Forest):")
print(f"   True Positives (TP):  {tp_rf:2d} - Versicolors korrekt als Versicolor erkannt")
print(f"   True Negatives (TN):  {tn_rf:2d} - Nicht-Versicolors korrekt als Nicht-Versicolor erkannt")
print(f"   False Positives (FP): {fp_rf:2d} - Nicht-Versicolors fälschlich als Versicolor erkannt (Fehlalarm)")
print(f"   False Negatives (FN): {fn_rf:2d} - Versicolors übersehen (verpasste Erkennung)")

## 📈 Schritt 7: Alle Metriken berechnen und verstehen

Jetzt berechnen wir alle wichtigen Metriken und verstehen, was sie bedeuten:

In [ ]:
# Alle Metriken für beide Modelle berechnen
def calculate_all_metrics(y_true, y_pred, model_name):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    # Zusätzliche Metriken
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
    
    print(f"\n📊 {model_name} - ALLE METRIKEN:")
    print("="*50)
    print(f"   Accuracy:     {accuracy:.3f} = {accuracy:.1%} aller Vorhersagen korrekt")
    print(f"   Precision:    {precision:.3f} = {precision:.1%} der 'Versicolor'-Vorhersagen stimmen")
    print(f"   Recall:       {recall:.3f} = {recall:.1%} aller Versicolors werden erkannt")
    print(f"   F1-Score:     {f1:.3f} = Harmonisches Mittel von Precision & Recall")
    print(f"   Specificity:  {specificity:.3f} = {specificity:.1%} der Nicht-Versicolors korrekt erkannt")
    print(f"   FPR:          {false_positive_rate:.3f} = {false_positive_rate:.1%} Fehlalarm-Rate")
    
    # Interpretation
    print(f"\n💡 PRAKTISCHE BEDEUTUNG:")
    if precision > 0.95:
        print(f"   🎯 Sehr hohe Precision: Wenn das Modell 'Versicolor' sagt, stimmt es fast immer!")
    if recall > 0.95:
        print(f"   🔍 Sehr hoher Recall: Das Modell übersieht fast keine Versicolors!")
    if f1 > 0.95:
        print(f"   ⚖️ Sehr hoher F1-Score: Exzellente Balance zwischen Precision & Recall!")
    
    return {
        'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1,
        'specificity': specificity, 'fpr': false_positive_rate
    }

# Metriken für beide Modelle
lr_metrics = calculate_all_metrics(y_test, y_pred, "LOGISTIC REGRESSION")
rf_metrics = calculate_all_metrics(y_test, y_pred_rf, "RANDOM FOREST")

# Detaillierter Classification Report
print(f"\n📋 DETAILLIERTER CLASSIFICATION REPORT:")
print("\nLogistic Regression:")
print(classification_report(y_test, y_pred, target_names=['Not Versicolor', 'Versicolor']))
print("\nRandom Forest:")
print(classification_report(y_test, y_pred_rf, target_names=['Not Versicolor', 'Versicolor']))

## 📉 Schritt 8: ROC-Kurve - Schwellenwert-Optimierung verstehen

**ROC-Kurve zeigt alle möglichen Schwellenwerte auf einen Blick!**

- **X-Achse:** False Positive Rate (Fehlalarm-Rate)
- **Y-Achse:** True Positive Rate = Recall (Erkennungsrate)

In [ ]:
# ROC-Kurven für beide Modelle
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, y_pred_proba)
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_pred_proba_rf)

auc_lr = roc_auc_score(y_test, y_pred_proba)
auc_rf = roc_auc_score(y_test, y_pred_proba_rf)

# ROC-Kurve plotten
plt.figure(figsize=(12, 5))

# ROC-Kurve
plt.subplot(1, 2, 1)
plt.plot(fpr_lr, tpr_lr, color='blue', lw=2, label=f'Logistic Regression (AUC = {auc_lr:.3f})')
plt.plot(fpr_rf, tpr_rf, color='green', lw=2, label=f'Random Forest (AUC = {auc_rf:.3f})')
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', label='Zufälliges Raten (AUC = 0.5)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (Fehlalarm-Rate)')
plt.ylabel('True Positive Rate (Recall)')
plt.title('ROC-Kurve: Schwellenwert-Performance', fontweight='bold')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)

# Precision-Recall Kurve
plt.subplot(1, 2, 2)
precision_lr, recall_lr, _ = precision_recall_curve(y_test, y_pred_proba)
precision_rf, recall_rf, _ = precision_recall_curve(y_test, y_pred_proba_rf)

plt.plot(recall_lr, precision_lr, color='blue', lw=2, label=f'Logistic Regression')
plt.plot(recall_rf, precision_rf, color='green', lw=2, label=f'Random Forest')
plt.axhline(y=np.mean(y_test), color='red', linestyle='--', label=f'Baseline (Zufall): {np.mean(y_test):.3f}')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall (Erkennungsrate)')
plt.ylabel('Precision (Genauigkeit der Vorhersagen)')
plt.title('Precision-Recall Kurve', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📊 ROC-AUC INTERPRETATION:")
print(f"   Logistic Regression AUC: {auc_lr:.3f}")
print(f"   Random Forest AUC:       {auc_rf:.3f}")
print(f"\n💡 AUC-BEWERTUNG:")
print(f"   0.90-1.00: Exzellent")
print(f"   0.80-0.90: Gut")
print(f"   0.70-0.80: Brauchbar")
print(f"   0.60-0.70: Schlecht")
print(f"   0.50-0.60: Sehr schlecht")
print(f"   0.50:      Wie Münzwurf (zufällig)")

# Bestes Modell bestimmen
best_model = "Logistic Regression" if auc_lr > auc_rf else "Random Forest"
best_auc = max(auc_lr, auc_rf)
print(f"\n🏆 BESTES MODELL: {best_model} mit AUC = {best_auc:.3f}")

## ⚖️ Schritt 9: Schwellenwert-Optimierung für verschiedene Anwendungsfälle

**Standard-Schwellenwert ist 0.5** - aber das ist nicht immer optimal!

Schauen wir uns verschiedene Szenarien an:

In [ ]:
def find_optimal_threshold(y_true, y_proba, objective='f1'):
    """Findet optimalen Schwellenwert für verschiedene Ziele"""
    thresholds = np.arange(0.1, 1.0, 0.01)
    best_score = 0
    best_threshold = 0.5
    
    scores = []
    for threshold in thresholds:
        y_pred_thresh = (y_proba >= threshold).astype(int)
        
        if objective == 'f1':
            score = f1_score(y_true, y_pred_thresh)
        elif objective == 'precision':
            score = precision_score(y_true, y_pred_thresh)
        elif objective == 'recall':
            score = recall_score(y_true, y_pred_thresh)
        
        scores.append(score)
        if score > best_score:
            best_score = score
            best_threshold = threshold
    
    return best_threshold, best_score, thresholds, scores

# Verschiedene Optimierungsziele
objectives = ['f1', 'precision', 'recall']
colors = ['blue', 'red', 'green']

plt.figure(figsize=(15, 5))

results = {}
for i, (obj, color) in enumerate(zip(objectives, colors)):
    threshold, score, thresholds, scores = find_optimal_threshold(y_test, y_pred_proba, obj)
    results[obj] = {'threshold': threshold, 'score': score}
    
    plt.subplot(1, 3, i+1)
    plt.plot(thresholds, scores, color=color, linewidth=2)
    plt.axvline(threshold, color='red', linestyle='--', alpha=0.7)
    plt.axvline(0.5, color='gray', linestyle=':', alpha=0.7, label='Standard (0.5)')
    plt.xlabel('Schwellenwert')
    plt.ylabel(f'{obj.upper()}-Score')
    plt.title(f'Optimierung für {obj.upper()}\nOptimal: {threshold:.2f} (Score: {score:.3f})', fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.legend()

plt.tight_layout()
plt.show()

print("⚖️ SCHWELLENWERT-OPTIMIERUNG:")
print("="*60)
for obj in objectives:
    print(f"{obj.upper():10}: Optimaler Schwellenwert = {results[obj]['threshold']:.3f}, Score = {results[obj]['score']:.3f}")

print(f"\nStandard:  Schwellenwert = 0.500")

# Praktische Szenarien
print(f"\n🎯 PRAKTISCHE ANWENDUNG:")
print(f"\n1. 🔬 BOTANISCHE FORSCHUNG: 'Keine Versicolor übersehen!'")
print(f"   → Optimiere für RECALL: Schwellenwert = {results['recall']['threshold']:.3f}")
print(f"   → Erkenne {results['recall']['score']:.1%} aller Versicolors")

print(f"\n2. 🏷️ AUTOMATISCHE ETIKETTIERUNG: 'Nur sicher markieren!'")
print(f"   → Optimiere für PRECISION: Schwellenwert = {results['precision']['threshold']:.3f}")
print(f"   → {results['precision']['score']:.1%} der 'Versicolor'-Labels sind korrekt")

print(f"\n3. ⚖️ AUSGEWOGENE ANWENDUNG: 'Beste Balance'")
print(f"   → Optimiere für F1-SCORE: Schwellenwert = {results['f1']['threshold']:.3f}")
print(f"   → F1-Score = {results['f1']['score']:.3f}")

## 🔬 Schritt 10: Fehleranalyse - Welche Blumen werden falsch klassifiziert?

Schauen wir uns die falsch klassifizierten Blumen genauer an:

In [ ]:
# Testdaten mit Vorhersagen analysieren
test_results = pd.DataFrame(X_test, columns=iris.feature_names)
test_results['True_Class'] = y_test
test_results['Predicted_Class'] = y_pred
test_results['Predicted_Proba'] = y_pred_proba
test_results['Correct'] = (y_test == y_pred)
test_results['True_Label'] = test_results['True_Class'].map({1: 'Versicolor', 0: 'Not_Versicolor'})
test_results['Pred_Label'] = test_results['Predicted_Class'].map({1: 'Versicolor', 0: 'Not_Versicolor'})

# Falsch klassifizierte Beispiele
false_predictions = test_results[~test_results['Correct']]

print(f"🔬 FEHLERANALYSE:")
print(f"   Gesamt getestet: {len(test_results)}")
print(f"   Korrekt: {test_results['Correct'].sum()}")
print(f"   Falsch: {len(false_predictions)}")
print(f"   Accuracy: {test_results['Correct'].mean():.1%}")

if len(false_predictions) > 0:
    print(f"\n❌ FALSCH KLASSIFIZIERTE BLUMEN:")
    print(false_predictions[['True_Label', 'Pred_Label', 'Predicted_Proba', 
                            'sepal length (cm)', 'sepal width (cm)', 
                            'petal length (cm)', 'petal width (cm)']].to_string(index=False))
    
    # Analyse der Fehlklassifikationen
    false_positives = false_predictions[false_predictions['True_Class'] == 0]
    false_negatives = false_predictions[false_predictions['True_Class'] == 1]
    
    print(f"\n📊 FEHLERTYPEN:")
    print(f"   False Positives: {len(false_positives)} (Nicht-Versicolor als Versicolor klassifiziert)")
    print(f"   False Negatives: {len(false_negatives)} (Versicolor als Nicht-Versicolor klassifiziert)")
    
    if len(false_positives) > 0:
        print(f"\n🚨 FALSE POSITIVE ANALYSE:")
        print(f"   Durchschnittliche Vorhersage-Wahrscheinlichkeit: {false_positives['Predicted_Proba'].mean():.3f}")
        print(f"   → Das Modell war sich nicht sehr sicher bei diesen Fehlern")
    
    if len(false_negatives) > 0:
        print(f"\n🚨 FALSE NEGATIVE ANALYSE:")
        print(f"   Durchschnittliche Vorhersage-Wahrscheinlichkeit: {false_negatives['Predicted_Proba'].mean():.3f}")
        print(f"   → Das Modell war sich nicht sehr sicher bei diesen Fehlern")
else:
    print(f"\n🎉 PERFEKTE KLASSIFIKATION!")
    print(f"   Alle Testblumen wurden korrekt klassifiziert!")

# Vertrauensanalyse
print(f"\n🎯 VERTRAUENSANALYSE:")
high_confidence = test_results[(test_results['Predicted_Proba'] > 0.9) | (test_results['Predicted_Proba'] < 0.1)]
medium_confidence = test_results[(test_results['Predicted_Proba'] >= 0.7) & (test_results['Predicted_Proba'] <= 0.9) | 
                                (test_results['Predicted_Proba'] >= 0.1) & (test_results['Predicted_Proba'] <= 0.3)]
low_confidence = test_results[(test_results['Predicted_Proba'] > 0.3) & (test_results['Predicted_Proba'] < 0.7)]

print(f"   Hohe Sicherheit (>90% oder <10%): {len(high_confidence)} Blumen, Accuracy: {high_confidence['Correct'].mean():.1%}")
print(f"   Mittlere Sicherheit (70-90% oder 10-30%): {len(medium_confidence)} Blumen, Accuracy: {medium_confidence['Correct'].mean():.1%}")
print(f"   Niedrige Sicherheit (30-70%): {len(low_confidence)} Blumen, Accuracy: {low_confidence['Correct'].mean():.1%}" if len(low_confidence) > 0 else "   Niedrige Sicherheit: 0 Blumen")

## 🎉 Zusammenfassung und wichtige Erkenntnisse

**Was wir heute gelernt haben:**

In [ ]:
# Finale Zusammenfassung
print("🎉 MODELLBEWERTUNG KOMPLETT!")
print("="*60)
print(f"\n📊 FINALE ERGEBNISSE:")
print(f"   Dataset: Iris (Versicolor vs. Rest)")
print(f"   Trainingssamples: {len(X_train)}")
print(f"   Testsamples: {len(X_test)}")
print(f"   Klassenverteilung: {np.mean(y_test):.1%} Versicolor")

print(f"\n🏆 BESTE PERFORMANCE (Test-Set):")
print(f"   Modell: {'Logistic Regression' if auc_lr > auc_rf else 'Random Forest'}")
print(f"   Accuracy: {max(lr_metrics['accuracy'], rf_metrics['accuracy']):.1%}")
print(f"   Precision: {max(lr_metrics['precision'], rf_metrics['precision']):.1%}")
print(f"   Recall: {max(lr_metrics['recall'], rf_metrics['recall']):.1%}")
print(f"   F1-Score: {max(lr_metrics['f1'], rf_metrics['f1']):.3f}")
print(f"   ROC-AUC: {max(auc_lr, auc_rf):.3f}")

print(f"\n✅ GELERNTE KONZEPTE:")
print(f"   ✓ Train/Test-Split mit Stratifizierung")
print(f"   ✓ Cross-Validation für robuste Performance-Schätzung")
print(f"   ✓ Data Leakage vermeiden mit Pipelines")
print(f"   ✓ Konfusionsmatrix interpretieren")
print(f"   ✓ Accuracy vs. Precision vs. Recall verstehen")
print(f"   ✓ ROC-Kurven und AUC bewerten")
print(f"   ✓ Schwellenwerte für verschiedene Anwendungsfälle optimieren")
print(f"   ✓ Modelle fair vergleichen")

print(f"\n🎯 WICHTIGSTE ERKENNTNISSE:")
print(f"   1. Versicolor ist schwieriger zu unterscheiden als Setosa - realistischeres Problem!")
print(f"   2. Beide Modelle zeigen gute aber nicht perfekte Performance")
print(f"   3. Schwellenwert-Optimierung kann die Performance für spezielle Anwendungen verbessern")
print(f"   4. Cross-Validation zeigt die Stabilität der Performance")
print(f"   5. Pipeline verhindert Data Leakage automatisch")

print(f"\n💡 FÜR DIE PRAXIS:")
print(f"   • Immer mehrere Metriken anschauen, nicht nur Accuracy")
print(f"   • Cross-Validation für robuste Schätzungen nutzen")
print(f"   • Schwellenwerte je nach Anwendungskontext optimieren")
print(f"   • Konfusionsmatrix zeigt, wo das Modell Schwächen hat")
print(f"   • Bei unbalancierten Daten: Precision-Recall wichtiger als ROC")

print("\n" + "="*60)
print("🎓 HERZLICHEN GLÜCKWUNSCH!")
print("Ihr könnt jetzt Klassifikationsmodelle professionell bewerten!")
print("="*60)